# Creando variables Campañas

Código para validar cruces y crear la variable respuesta

In [1]:
import boto3
import pandas as pd
import numpy as np
import pickle
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
import pandas_profiling
import os
from datetime import datetime, timedelta
import dateutil.relativedelta
import gc

pd.set_option('display.max_columns',None)

In [2]:
import s3fs
import pyarrow.parquet as pq

def spark_read_parquet(s3_url: str, **args):
    fs = s3fs.S3FileSystem()
    # Leyendo base
    dataset = pq.ParquetDataset(s3_url, filesystem=fs)
    table = dataset.read()
    dataframe = table.to_pandas()

    del dataset, table

    return dataframe

def list_files(s3_path):
    """
    Función para listar los archivos de un folder en s3
    :param s3_path: Ruta en S3 en la cual se quiere ver
                    los archivos disponibles
    :return: Lista de archivos encontrados
    """
    s3 = boto3.resource("s3")
    bucket = s3_path.split("//")[1].split("/")[0]
    my_bucket = s3.Bucket(bucket)
    prefix = "/".join(s3_path.split("//")[1].split("/")[1:])
    list_obj = []
    for object_summary in my_bucket.objects.filter(Prefix=prefix):
        obj_x = object_summary.key.replace(prefix + "/", "").split("/")[0]
        cond_1 = not any([x == obj_x for x in list_obj])
        cond_2 = not (object_summary.key[-9:] == "_$folder$")
        if cond_1 & cond_2:
            list_obj.append(obj_x)
    list_obj = [x for x in list_obj if not x == prefix]
    list_obj = [x for x in list_obj if not x == ""]
    return list_obj

In [3]:
path_mf = r's3://data-bpop-dev-sandbox/estandarizado/transacciones/tarjeta-credito-masterfile'

def trx_masterfile(fecha):
    """
    Descripcion; Función para hacer tratamiento masterfile para una única tabla
    Parámetros: fecha (numérico) en formato yyyymm
    Retorno: data masterfile con el tratamiento adecuado de una sóla tabla
    ejemplo: trx_masterfile(202006)
    """
    #fecha = 202108
    day = "01"
    fecha_hoy = str(fecha) + day
    fecha_total = int(fecha_hoy)  ###Con este valor se realiza la busqueda sobre las tablas acumuladas
    ##A continuacion, se detallan algunas fechas
    start = datetime.strptime(str(fecha_hoy), '%Y%m%d').strftime('%m/%d/%Y')
    start = datetime.strptime(start, "%m/%d/%Y")
    ##Para obtener la ficha inicial y la fecha final dia 1 y dia final del mes
    fecha_inicial = start
    fecha_final = start + timedelta(days=35)  ##un mes despues
    fecha_final = fecha_final - timedelta(days=fecha_final.day)
    ##Para convrtir en formato fecha
    fecha_inicial = fecha_inicial.strftime('%Y%m%d')
    fecha_final = fecha_final.strftime('%Y%m%d')
    ##A cotninuación, se define el nombre de la tabla
    ##Información importante: !!!!
    ## Dado que aún no se encuentra estandarizado los datos, entonces, se define el nombre de las tablas
    ## de la siguiente forma:

    if (fecha_total >= 20201001):
        file = 'transacciones_tarjeta-credito-masterfile_masterfile_M' + str(fecha)
    else:
        if (fecha_total >= 20200701 and fecha_total <= 20200930):
            file = "transacciones_tarjeta-credito-masterfile_masterfile_H20200701-20200930"
        else:
            if (fecha_total >= 20200101 and fecha_total <= 20200630):
                file = "transacciones_tarjeta-credito-masterfile_masterfile_H20200101-20200630"
            else:
                if (fecha_total >= 20190701 and fecha_total <= 20191231):
                    file = "transacciones_tarjeta-credito-masterfile_masterfile_H20190701-20191231"
                else:
                    if (fecha_total >= 20190101 and fecha_total <= 20190630):
                        file = "transacciones_tarjeta-credito-masterfile_masterfile_H20190101-20190630"
                    else:
                        file = "transacciones_tarjeta-credito-masterfile_masterfile_H20180701-20181231"
    ##Para determinar la ruta fuente de la tabla transaccional

    path = path_mf
    #list_files(path)
    #Para definir al ruta completa

    #A continuación,se construye una función que determina si la tabla existe para la fecha dada
    if file in list_files(path):
        print("Si existe la data trx masterfile para la fecha " + str(fecha))
        ##A continuación, se realiza una lectura de la data transaccional repgesco
        data = spark_read_parquet(path + '/' + file)
        # data.show()
        # vector_columns = data.columns
        # len(vector_columns)
        # data.count()
        ##Para seleccionar únicamente las transacciones asociadas a compras y avances
        ##Para seleccionar únicamente las transacciones asociadas a compras y avances
        vector = ['2021', '2043', '5002', '5006', '5013', '5015',
                  '5022', '8001', '8003', '8004', '8006', '8007',
                  '8008', '8109', '2332', '2321', '2362', '8511',
                  '8509', '8511', '2024', '2025', '2026', '2041',
                  '2324', '2334', '2363', '5075', '2333', '2328',
                  '8011', '8017']
        data = data[data.cod_unico_trx.isin(vector)]
        data.columns = data.columns.str.upper()
        data["FECHA_COMPROBANTE"] = data["FECHA_PROCESO"]
        old_names = ['TIPO_ID',
                     'IDENTIFICACION_CLIENTE',
                     'NRO_CUENTA',
                     'NUMERO_TARJETA',
                     'COD_UNICO_TRX',
                     'FECHA_COMPROBANTE',
                     'VALOR_TOTAL',
                     'NOMBRE_COMERCIO',
                     'CIUDAD_COMERCIO',
                     'FECHA_TRANSACCION',
                     'COD_MONEDA_ORIGEN',
                     'CUOTAS',
                     'FECHA_PROCESO',
                     'MCC',
                     'BIN']
        data = data[old_names]
        ##A continuación, se realiza el cambio de nombre de variables
        new_names = ['TIPO_ID',
                     'IDENTIFICACION_CLIENTE',
                     'NRO_CUENTA',
                     'NUMERO_TARJETA',
                     'COD_TRX',
                     'FECHA_COMPROBANTE',
                     'VALOR_TOTAL',
                     'NOMBRE_COMERCIO',
                     'CIUDAD_COMERCIO',
                     'FECHA_TRANSACCION',
                     'COD_MONEDA_ORIGEN',
                     'CUOTAS',
                     'FECHA_PROCESO',
                     'MCC',
                     'BIN']
        mapping = dict(zip(old_names, new_names))
        data = data.rename(columns=mapping)
        ##Para construir la variable tipo de TC
        data["Tipo_TC"] = np.where(data["BIN"] == 474638, "Express",
                               np.where(data["BIN"] == 434761, "La 14",
                               np.where(data["BIN"] == 409648, "La 14",
                               np.where(data["BIN"] == 409649, "La 14",
                               "Tradicional"))))
        ##Seleccionar únicamente tarjetas de crédito tradicional
        data = data[data["Tipo_TC"] == "Tradicional"]
        ##Para no seleccionar las tarjetas de creditos empresariales
        data = data[data["BIN"] != 454476]
        ##Para seleccionar las variables importantes
        variables_importantes = ['TIPO_ID',
                                 'IDENTIFICACION_CLIENTE',
                                 'NRO_CUENTA',
                                 'NUMERO_TARJETA',
                                 'FECHA_TRANSACCION',
                                 'VALOR_TOTAL',
                                 'NOMBRE_COMERCIO',
                                 'CUOTAS',
                                 'Tipo_TC',
                                 "MCC"]
        data = data[variables_importantes]
        data['FECHA_TRANSACCION'] = pd.to_datetime(data['FECHA_TRANSACCION'], format='%Y-%m-%d')
        ##Para seleccionar las transacciones asociadas al mes de interés
        fecha_inicial = fecha_inicial[0:4] + "-" + fecha_inicial[4:6] + "-" + fecha_inicial[6:9] + " 00:00:00"
        fecha_final = fecha_final[0:4] + "-" + fecha_final[4:6] + "-" + fecha_final[6:9] + " 00:00:00"
        data = data[(data['FECHA_TRANSACCION'] >= fecha_inicial) & (data['FECHA_TRANSACCION'] <= fecha_final)]
        data["periodo"] = fecha
        data.columns = data.columns.str.lower()
        data = data.drop_duplicates()
        # data.show()
        # express_la_14 = data.filter(col("Tipo_TC") != "Tradicional")
        # express_la_14.show()
        # express_la_14.count()
        # data.count()
        print("Se ha realizado el correcto tratamiento de la data masterfile periodo " + str(fecha))
        # data.show()
    else:
        print("No existe la data trx masterfile para la fecha " + str(fecha))
        data = []
    
    return data


In [4]:
def func_activa(base,date,n_mes,nom_col):
    if len(base)>0:
        date_fin = date + relativedelta(months=n_mes)

        data_act_temp = data_act.loc[(data_act['mes_activacion']>=date) & (data_act['mes_activacion']<=date_fin),['cedula','numero_cuenta','fecha_activacion','mes_activacion']].drop_duplicates()
        ids_mes = data_act_temp['cedula'].unique()
        
        base.loc[base['cedula'].isin(ids_mes),'venta'] = 1    
        base.loc[base['venta'].isnull(),'venta'] = 0
        
        base.loc[:,nom_col] = 1   
        #base = pd.merge(base,data_act_temp,on=['cedula','ID_CLIENTE'],how='left')
        base = pd.merge(base,data_act_temp,on=['cedula'],how='left')
    
    return base

# Cargando bases de campañas

## Oficinas

In [5]:
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

def get_s3_files(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"] for file in objects]
    return files


def get_s3_parquet(s3_url, flag):
    """
    s3_url: path donde se desea buscar
    flag: permite identificar por cual camino entrar: 1: buscar archivo especifico, 0: buscar carpeta, ejemplo las std
    """
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    print('bucket_last_file: ', bucket)
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    print('key_last_file: ', key)
#     if flag == "1":
#         objects = _key_files__list(s3, bucket, key)
#         #files = ["s3://" + bucket + "/" + file["Key"] for file in objects]
#         files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if
#                  file["Key"].split("/")[-1] == "_SUCCESS"]
#     elif flag == "0":
#         objects = _key_files__list(s3, bucket, key)
#         files = ["s3://" + bucket + "/" + file["Prefix"] for file in objects]
#     else:
#         files = []
    files = _key_files__list(s3, bucket, key)
    return files

In [6]:
path = 's3://data-bpop-dev-refined/gestion-campanas/preaprobado-tc/'
bucket = 'data-bpop-dev-refined'

In [7]:
objects = get_s3_parquet(path,0)
files = ["s3://" + bucket + "/" + file["Key"] for file in objects]

path_file = files[-1]

bucket_last_file:  data-bpop-dev-refined
key_last_file:  gestion-campanas/preaprobado-tc/


In [8]:
data_temp = pd.read_csv(path_file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str}) 

In [9]:
data_temp.head()

,date,act_date,base_name,clientid,office_code
0,2020-07-03,20200630,CALL_ATENTO_20200702.csv,10002878,470
1,2020-07-03,20200630,CALL_ATENTO_20200702.csv,10002898,660
2,2020-07-03,20200630,CALL_ATENTO_20200702.csv,10009775,470
3,2020-07-03,20200630,CALL_ATENTO_20200702.csv,10012869,470
4,2020-07-03,20200630,CALL_ATENTO_20200702.csv,10020225,470


In [10]:
data_temp.tail()

,date,act_date,base_name,clientid,office_code
278861,2022-02-03,20220201,OFICINA_BOGOTA_20220202.csv,9514630,270
278862,2022-02-03,20220201,OFICINA_BOGOTA_20220202.csv,9530597,270
278863,2022-02-03,20220201,OFICINA_BOGOTA_20220202.csv,9651735,270
278864,2022-02-03,20220201,OFICINA_BOGOTA_20220202.csv,9653868,88
278865,2022-02-03,20220201,OFICINA_BOGOTA_20220202.csv,9778378,460


In [11]:
data_temp = data_temp.sort_values(['base_name','date','clientid'])
print(data_temp.shape)
data_temp = data_temp.groupby(['clientid','base_name']).first()
print(data_temp.shape)
data_temp.reset_index(inplace=True)

(278866, 5)
(237134, 3)


In [12]:
## Filtrar express-la14

data_temp = data_temp.loc[~data_temp['base_name'].str.contains('EXPRES_LA14')]
data_temp = data_temp.rename(columns={'clientid':'NUM_DOC'})

## Separando:
## Oficinas

data_crm = data_temp.loc[~data_temp['base_name'].str.contains('OFICINA_BOGOTA')]
## tipo campaña
data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
data_crm['tipo_camp'] = 'of-preaprobados'
# Casting variables
data_crm['FECHA'] = pd.to_datetime(data_crm['date'],format='%Y-%m-%d')
data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=25, 
                                                              data_crm['FECHA']+timedelta(days=10),
                                                              data_crm['FECHA']),format='%Y-%m-%d')
data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')
data_crm1 = data_crm.copy()

## Atento

data_vend = data_temp.loc[~data_temp['base_name'].str.contains('CALL_ATENTO')]
# Casting variables
data_vend['TIPO CAMPAÑA'] = 'Preaprobados'
data_vend['tipo_camp'] = 'at-preaprobados'
data_vend['FECHA'] = pd.to_datetime(data_vend['date'],format='%Y-%m-%d')
data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
data_vend1 = data_vend.copy()

/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/home/jupyter/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[r

# Ya no

In [13]:
input_paths = {'crm':[
#                       's3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-crm_M202103.csv',
#                      's3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-crm_H20200801-20210201.csv',
                       's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv',
                      's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv',
                      's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv'
                    ],
               'vendors':[
#                         's3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-vendors_H20201201-20210301.csv',
#                         's3://data-bpop-dev-sandbox/landing/tmp/tmpg_campanas-tc-nuevas-proc-vendors_H20200801-20210101.csv',
                          's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv',
                         's3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv'
                ]}

In [14]:
obj = 'crm'

files = input_paths[obj]

for ix, file in enumerate(files):
    print('File:', file)
    
    data_temp = pd.read_csv(file, sep='|', dtype={'ID_CLIENTE':str,'NUM_DOC':str}) 
    
    if ix == 0:
        data_crm = data_temp
    else:
        data_crm = pd.concat([data_crm,data_temp], ignore_index=True)
        
    del data_temp

File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20200228-20200601.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190801-20200201.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-crm/campanas-tc-nuevas-proc-crm_H20190104-20190702.csv


In [15]:
data_crm.head()

,NUM_DOC,ID_CLIENTE,FECHA
0,11331659,728652315426013901,28/02/2020
1,20287021,971552296816188702,28/02/2020
2,32445739,227552309591472601,28/02/2020
3,2999236,122252294553987002,28/02/2020
4,5710110,758052310452202501,28/02/2020


In [16]:
## tipo campaña
data_crm['TIPO CAMPAÑA'] = 'Preaprobados'
data_crm['tipo_camp'] = 'of-preaprobados'

In [17]:
# Casting variables
data_crm['FECHA'] = pd.to_datetime(data_crm['FECHA'],format='%d/%m/%Y')
data_crm['mes_campaña'] = pd.to_datetime(np.where(data_crm['FECHA'].dt.day>=28, 
                                                              data_crm['FECHA']+timedelta(days=10),
                                                              data_crm['FECHA']),format='%Y-%m-%d')
data_crm['mes_campaña2'] = pd.to_datetime(data_crm['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')
data_crm['mes_campaña'] = pd.to_datetime(data_crm['mes_campaña'].dt.strftime('%Y-%m'),format='%Y-%m')

In [18]:
data_crm = pd.concat([data_crm1,data_crm], ignore_index=True)

## Atento

In [19]:
obj = 'vendors'

files = input_paths[obj]

In [20]:
obj = 'vendors'

files = input_paths[obj]

for ix, file in enumerate(files):
    print('File:', file)
    
    data_temp = pd.read_csv(file, sep='|', encoding='ISO8859-1',quoting=1, dtype={'ID_CLIENTE':str,'NUM_DOC':str})
    if ix == 0:
        data_vend = data_temp.copy()
    else:
        data_vend = pd.concat([data_vend,data_temp], ignore_index=True)
        
    del data_temp

File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20200401-20200601.csv
File: s3://adl-refined-dev-popular/data_orig/campanas/tc-nuevas-proc-vend/campanas-tc-nuevas-proc-vendors_H20190101-20200301.csv


In [21]:
data_vend.head()

,TIP_DOC,NUM_DOC,ID_CLIENTE,CUPO_APROBADO,FECHA,TIPO CAMPAÑA
0,1.0,7332107,234952295182773302,NaN,01/05/2020,Perfilados Activos
1,1.0,2919898,268152294536287301,NaN,01/05/2020,Perfilados Activos
2,1.0,41386486,102652313123946401,NaN,01/05/2020,Perfilados Activos
3,1.0,19233476,248852314626474002,NaN,01/05/2020,Perfilados Activos
4,1.0,399382,183452294188281002,NaN,01/05/2020,Perfilados Activos


In [22]:
data_vend['TIPO CAMPAÑA'].value_counts()

Preaprobados            61502
Perfilados Activos      26931
Perfilados inactivos    16072
Perfilados Inactivos     1000
Name: TIPO CAMPAÑA, dtype: int64

In [23]:
# Casting variables
data_vend['tipo_camp'] = np.where(data_vend['TIPO CAMPAÑA'] == 'Preaprobados','at-preaprobados','at-perfilados')
data_vend['FECHA'] = pd.to_datetime(data_vend['FECHA'], format='%d/%m/%Y')
data_vend['mes_campaña'] = pd.to_datetime(data_vend['FECHA'].dt.strftime('%Y-%m'),format='%Y-%m')

In [24]:
data_vend = pd.concat([data_vend1,data_vend], ignore_index=True)

### Uniendo bases 

In [25]:
cols = ['NUM_DOC', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp']

In [26]:
data_vend.columns = data_vend.columns.str.strip()

In [27]:
#data_camp = pd.concat([data_crm[cols1], data_vend[cols2]], ignore_index=True)[cols2]
data_camp = pd.concat([data_crm[cols], data_vend[cols]], ignore_index=True)[cols]
print(data_camp.shape)
data_camp 

(339489, 5)


,NUM_DOC,FECHA,mes_campaña,TIPO CAMPAÑA,tipo_camp
0,93442,2021-09-06,2021-09-01,Preaprobados,of-preaprobados
1,96681,2021-04-30,2021-05-01,Preaprobados,of-preaprobados
2,96681,2021-09-06,2021-09-01,Preaprobados,of-preaprobados
3,98362,2020-07-03,2020-07-01,Preaprobados,of-preaprobados
4,100090,2021-04-30,2021-05-01,Preaprobados,of-preaprobados
...,...,...,...,...,...
339484,17007499,2020-03-01,2020-03-01,Preaprobados,at-preaprobados
339485,17112493,2020-03-01,2020-03-01,Preaprobados,at-preaprobados
339486,3362730,2020-03-01,2020-03-01,Preaprobados,at-preaprobados
339487,3430980,2020-03-01,2020-03-01,Preaprobados,at-preaprobados


In [28]:
data_camp['NUM_DOC'] = data_camp['NUM_DOC'].astype(np.int64)
data_camp['FECHA'] = pd.to_datetime(data_camp['FECHA'])
data_camp['mes_campaña'] = pd.to_datetime(data_camp['mes_campaña'])

In [29]:
data_camp.count()

NUM_DOC         339489
FECHA           339489
mes_campaña     339489
TIPO CAMPAÑA    339489
tipo_camp       339489
dtype: int64

# Conteos

In [30]:
data_camp['mes_campaña'].value_counts().sort_index()

2019-01-01      457
2019-02-01     6370
2019-03-01     5001
2019-04-01     6662
2019-05-01     6761
2019-06-01     6749
2019-07-01     5857
2019-08-01     7045
2019-09-01     7212
2019-10-01     6952
2019-11-01     7235
2019-12-01     5472
2020-01-01     7332
2020-02-01     8338
2020-03-01    11126
2020-04-01     7503
2020-05-01    12749
2020-06-01     9727
2020-07-01    11755
2020-08-01    12903
2020-09-01    13456
2020-10-01    14364
2020-11-01    11712
2020-12-01    11789
2021-01-01     9975
2021-02-01    12056
2021-03-01     8712
2021-04-01    13412
2021-05-01     9336
2021-06-01    10952
2021-07-01     9391
2021-08-01     8830
2021-09-01     9955
2021-10-01     2346
2021-11-01     9977
2021-12-01     9967
2022-01-01     9126
2022-02-01    10927
Name: mes_campaña, dtype: int64

In [31]:
pd.crosstab(data_vend['mes_campaña'] ,data_vend['TIPO CAMPAÑA'])

TIPO CAMPAÑA,Perfilados Activos,Perfilados Inactivos,Perfilados inactivos,Preaprobados
mes_campaña,,,,
2019-02-01,1500,0,2439,777
2019-03-01,2000,0,1250,1751
2019-04-01,2000,0,1250,1753
2019-05-01,2002,0,1252,1752
2019-06-01,2002,0,1251,1756
2019-07-01,2001,0,172,1756
2019-08-01,2000,0,1250,1751
2019-09-01,2000,0,1250,1750
2019-10-01,2001,0,1251,1443


In [32]:
pd.crosstab(data_crm['mes_campaña'] ,data_crm['TIPO CAMPAÑA'])

TIPO CAMPAÑA,Preaprobados
mes_campaña,
2019-01-01,457
2019-02-01,1654
2019-04-01,1659
2019-05-01,1755
2019-06-01,1740
2019-07-01,1928
2019-08-01,2044
2019-09-01,2212
2019-10-01,2257


## Activaciones

In [33]:
path_act = ['s3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H202003-202005.csv',
            's3://adl-refined-dev-popular/data_orig/productos/activo-tarjeta-credito-activadas/productos_activos-tarjeta-credito-activadas_H20180101-20200327.csv'
           ]
path = 's3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/Productos_Activo-Tarjeta-credito-nueva_Masterfile_D'

In [34]:
data_act2 = pd.read_csv(path_act[0], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
data_act2 = data_act2.drop_duplicates()

print(data_act2.shape)

(3157, 6)


In [35]:
data_act1 = pd.read_csv(path_act[1], sep = '|', dtype={'ID_CLIENTE':str,'Numero_cuenta':str,'Identificacion':str})
data_act1 = data_act1.drop_duplicates()

print(data_act1.shape)

(40411, 6)


In [36]:
## Uniendo bases
data_act2['fecha_activacion'] = data_act2['fecha_activacion'].str.replace('/20','/2020')
data_act1 = pd.concat([data_act2, data_act1], ignore_index=True)

# Casting variables
data_act1['fecha_activacion'] = pd.to_datetime(data_act1['fecha_activacion'],format='%d/%m/%Y')
data_act1['mes_activacion'] = pd.to_datetime(data_act1['fecha_activacion'].dt.strftime('%Y-%m'),format='%Y-%m')

del data_act2

In [37]:
data_act1.columns = data_act1.columns.str.lower()
data_act1.columns

Index(['numero_cuenta', 'numero_tarjeta', 'identificacion',
       'tipo_identificacion', 'fecha_activacion', 'id_cliente',
       'mes_activacion'],
      dtype='object')

In [38]:
import boto3
import boto3.session

def _key_existing_size__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    for obj in response.get("Contents", []):
        if obj["Key"] == key:
            return obj["Size"]
        
def _key_files__list(client, bucket, key):
    """return the key's size if it exist, else None"""
    response = client.list_objects_v2(Bucket=bucket, Prefix=key)
    return response.get("Contents", [])

def parquet_exists(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)+ "/_SUCCESS"
    size = _key_existing_size__list(s3, bucket, key)
    if size is not None:
        print("File exists:", s3_url)
        return True
    return False

def get_s3_parquets(s3_url):
    session = boto3.session.Session()
    s3 = session.client("s3")
    bucket = s3_url.split("/")[2]
    key = s3_url.split("/")[3:]
    key = "/".join(key)
    objects = _key_files__list(s3, bucket, key)
    files = ["s3://" + bucket + "/" + file["Key"][:-9] for file in objects if file["Key"].split("/")[-1]=="_SUCCESS"]
    return files

In [39]:
s3_url = 's3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva'
files = get_s3_parquets(s3_url)

In [40]:
for ix, file in enumerate(files):
    print('File:', file)
    data_temp = spark_read_parquet(file)
    if ix == 0:
        data_act = data_temp
    else:
        data_act = pd.concat([data_act,data_temp], ignore_index=True)
        
    del data_temp       

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200511
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200512
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200513
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200514
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200518
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200519
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200727
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200728
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200729
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200730
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200731
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200801
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200924
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200925
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200926
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200927
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200928
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20200929
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201121
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201122
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201123
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201124
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201125
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20201126
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210118
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210119
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210120
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210121
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210122
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210123
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210317
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210318
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210319
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210320
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210321
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210322
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210513
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210514
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210515
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210516
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210517
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210518
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210710
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210711
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210712
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210808
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210809
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20210810
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20211001
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20211002
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20211003
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20211004
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20211005
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva_masterfile_D20211006
File: s3://data-bpop-dev-sandbox/estandarizado/productos/activo-tarjeta-credito-nueva/productos_activo-tarjeta-credito-nueva

In [41]:
data_act.columns

Index(['tp_informe', 'tipo_id_cliente', 'id_cli', 'numero_tarjeta',
       'numero_cuenta', 'amrm_primary_acct', 'nombre_completo', 'fecha_realce',
       'fecha_apertura_cta', 'codigo_oficina', 'fecha_activacion_plastico',
       'calidad_cliente', 'ambs_rel_nbr', 'bloqueo_cuenta',
       'fecha_bloqueo_cuenta', 'bloqueo_cartera', 'bloqueo_plastico',
       'fecha_bloqueo_plastico', 'estado_cuenta', 'cupo', 'emblem',
       'nombre_emblem', 'pct', 'motivo', 'sk_cliente', 'id_cliente', 'cont_id',
       'tipo_id', 'tipo_id_homologado'],
      dtype='object')

In [42]:
data_act['fecha_activacion_plastico'].head(15)

0     0020-05-04
1     0020-05-04
2     0020-05-05
3     0020-05-03
4     0020-05-02
5     0020-05-04
6     0020-05-02
7           None
8     0020-05-03
9     0020-05-05
10    0020-05-05
11    0020-05-03
12    0018-12-26
13    0020-05-03
14    0020-05-03
Name: fecha_activacion_plastico, dtype: object

In [43]:
str(data_act['fecha_activacion_plastico'][0])[:2]

'00'

In [44]:
# Casting variables
data_act['fecha_activacion_plastico'] = pd.Series([str(x).replace('00','20') if '00' in str(x)[:2] else str(x) for x in data_act['fecha_activacion_plastico']])
data_act['fecha_activacion_plastico'] = pd.to_datetime(data_act['fecha_activacion_plastico'], errors='coerce')
data_act['mes_activacion'] = pd.to_datetime(data_act['fecha_activacion_plastico'].dt.strftime('%Y-%m'),format='%Y-%m')

In [45]:
## Renombrando variables
data_act1.columns = data_act1.columns.str.lower()
data_act1 = data_act1.rename(columns={'identificacion':'cedula'})
data_act = data_act.rename(columns={'id_cli':'cedula',
                                   'tipo_id_cliente':'tipo_identificacion',
                                   'fecha_activacion_plastico':'fecha_activacion'})
data_camp = data_camp.rename(columns={'NUM_DOC':'cedula'})

In [46]:
data_act = data_act[['numero_cuenta', 'numero_tarjeta', 'cedula',
       'tipo_identificacion', 'fecha_activacion', 'id_cliente',
       'mes_activacion']]

In [47]:
## Uniendo bases
data_act = pd.concat([data_act1, data_act], ignore_index=True)
data_act = data_act.drop_duplicates()

del data_act1

In [48]:
## Cast variables
data_act['id_cliente'] = data_act['id_cliente'].fillna('-9999').astype(np.int64)
data_act['tipo_identificacion'] = data_act['tipo_identificacion'].fillna('-9999').astype(np.int64)
data_act['cedula'] = data_act['cedula'].astype(np.int64)

In [49]:
data_act.head(5)

,numero_cuenta,numero_tarjeta,cedula,tipo_identificacion,fecha_activacion,id_cliente,mes_activacion
0,NaN,539168******0829,160853,2,2020-03-30,392752305340136601,2020-03-01
1,NaN,420559******1208,160853,2,2020-03-30,392752305340136601,2020-03-01
2,NaN,420559******4257,195017,1,2020-05-06,264252317716356802,2020-05-01
3,NaN,420559******4961,208056,1,2020-05-22,295352314103188701,2020-05-01
4,NaN,539238******8374,208056,1,2020-05-22,295352314103188701,2020-05-01


In [50]:
data_act.count()

numero_cuenta          137851
numero_tarjeta         141008
cedula                 141008
tipo_identificacion    141008
fecha_activacion       135955
id_cliente             141008
mes_activacion         135955
dtype: int64

In [51]:
data_act.dtypes

numero_cuenta                  object
numero_tarjeta                 object
cedula                          int64
tipo_identificacion             int64
fecha_activacion       datetime64[ns]
id_cliente                      int64
mes_activacion         datetime64[ns]
dtype: object

In [52]:
str_dates = ['2019-01', '2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
          '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
          '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08', '2021-09',
            '2021-10', '2021-11','2021-12','2022-01']
date = str_dates[0]
date = pd.to_datetime(date,format='%Y-%m')
date

Timestamp('2019-01-01 00:00:00')

In [53]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

str_dates = ['2019-01', '2019-02','2019-03','2019-04','2019-05','2019-06','2019-07','2019-08','2019-09','2019-10','2019-11',
          '2019-12','2020-01','2020-02','2020-03','2020-04','2020-05','2020-06','2020-07','2020-08','2020-09','2020-10',
          '2020-11','2020-12','2021-01','2021-02','2021-03','2021-04','2021-05','2021-06','2021-07','2021-08', '2021-09',
            '2021-10', '2021-11','2021-12','2022-01']

n_mes = 3 # numero de meses

for ix, date in enumerate(str_dates):
    print('Fecha:',date)
    date = pd.to_datetime(date,format='%Y-%m')
    nom_col = 'camp_'+date.strftime('%Y%m')
    
    base = data_camp[data_camp['mes_campaña']==date]

    if ix == 0:
        base_final = func_activa(base,date,n_mes,nom_col)
        del base
    else:
        base_temp = func_activa(base,date,n_mes,nom_col)
        base_final = pd.concat([base_final,base_temp],ignore_index=True)
        
        del base, base_temp
        

Fecha: 2019-01
Fecha: 2019-02
Fecha: 2019-03
Fecha: 2019-04
Fecha: 2019-05
Fecha: 2019-06
Fecha: 2019-07
Fecha: 2019-08
Fecha: 2019-09
Fecha: 2019-10


/home/jupyter/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1599: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
/home/jupyter/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/home/jupyter/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

Fecha: 2019-11
Fecha: 2019-12
Fecha: 2020-01
Fecha: 2020-02
Fecha: 2020-03
Fecha: 2020-04
Fecha: 2020-05
Fecha: 2020-06
Fecha: 2020-07
Fecha: 2020-08
Fecha: 2020-09
Fecha: 2020-10
Fecha: 2020-11
Fecha: 2020-12
Fecha: 2021-01
Fecha: 2021-02
Fecha: 2021-03
Fecha: 2021-04
Fecha: 2021-05
Fecha: 2021-06
Fecha: 2021-07
Fecha: 2021-08
Fecha: 2021-09
Fecha: 2021-10
Fecha: 2021-11
Fecha: 2021-12
Fecha: 2022-01


In [54]:
from dateutil.relativedelta import relativedelta
n_mes=3
date_fin = date + relativedelta(months=n_mes)
date_fin

Timestamp('2022-04-01 00:00:00')

In [55]:
date

Timestamp('2022-01-01 00:00:00')

In [56]:
base_final.columns

Index(['cedula', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp', 'venta',
       'camp_201901', 'numero_cuenta', 'fecha_activacion', 'mes_activacion',
       'camp_201902', 'camp_201903', 'camp_201904', 'camp_201905',
       'camp_201906', 'camp_201907', 'camp_201908', 'camp_201909',
       'camp_201910', 'camp_201911', 'camp_201912', 'camp_202001',
       'camp_202002', 'camp_202003', 'camp_202004', 'camp_202005',
       'camp_202006', 'camp_202007', 'camp_202008', 'camp_202009',
       'camp_202010', 'camp_202011', 'camp_202012', 'camp_202101',
       'camp_202102', 'camp_202103', 'camp_202104', 'camp_202105',
       'camp_202106', 'camp_202107', 'camp_202108', 'camp_202109',
       'camp_202110', 'camp_202111', 'camp_202112', 'camp_202201'],
      dtype='object')

In [57]:
# Casting variables
base_final = base_final[base_final['cedula'].notnull()]

#base_final.rename(columns={' CUPO_APROBADO ':'CUPO_APROBADO'}, inplace=True)
#base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].str.strip()
#base_final['CUPO_APROBADO'] = [str(x).replace('.','') for x in base_final['CUPO_APROBADO']]
#base_final['CUPO_APROBADO'] = base_final['CUPO_APROBADO'].astype('float')

cols = ['camp_201901','camp_201902','camp_201903','camp_201904','camp_201905','camp_201906',
            'camp_201907','camp_201908','camp_201909','camp_201910','camp_201911','camp_201912',
            'camp_202001','camp_202002','camp_202003','camp_202004','camp_202005','camp_202006',
        'camp_202007','camp_202008','camp_202009','camp_202010','camp_202011','camp_202012',
        'camp_202101','camp_202102','camp_202103','camp_202104','camp_202105','camp_202106','camp_202107',
        'camp_202108', 'camp_202109', 'camp_202110', 'camp_202111', 'camp_202112', 'venta']
                                                                              
base_final.loc[:,cols] = base_final[cols].fillna(0)

In [58]:
# Casting variables
base_final['fecha_activacion'] = pd.to_datetime(base_final['fecha_activacion'], format='%Y-%m%d')
base_final['mes_activacion'] = pd.to_datetime(base_final['mes_activacion'], format='%Y-%m%d')

base_final

,cedula,FECHA,mes_campaña,TIPO CAMPAÑA,tipo_camp,venta,camp_201901,numero_cuenta,fecha_activacion,mes_activacion,camp_201902,camp_201903,camp_201904,camp_201905,camp_201906,camp_201907,camp_201908,camp_201909,camp_201910,camp_201911,camp_201912,camp_202001,camp_202002,camp_202003,camp_202004,camp_202005,camp_202006,camp_202007,camp_202008,camp_202009,camp_202010,camp_202011,camp_202012,camp_202101,camp_202102,camp_202103,camp_202104,camp_202105,camp_202106,camp_202107,camp_202108,camp_202109,camp_202110,camp_202111,camp_202112,camp_202201
0,79711820,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,20896135,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,1.0,1.0,5361706003729575037,2019-02-05,2019-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,20896135,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,1.0,1.0,4506584002622716431,2019-02-05,2019-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,41527303,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,79491343,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334817,1111747929,2022-01-05,2022-01-01,Preaprobados,at-preaprobados,0.0,0.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
334818,1111761735,2022-01-05,2022-01-01,Preaprobados,at-preaprobados,0.0,0.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
334819,1111764595,2022-01-05,2022-01-01,Preaprobados,at-preaprobados,0.0,0.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
334820,1122782607,2022-01-05,2022-01-01,Preaprobados,at-preaprobados,0.0,0.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [59]:
base_final.count()

cedula              334822
FECHA               334822
mes_campaña         334822
TIPO CAMPAÑA        334822
tipo_camp           334822
venta               334822
camp_201901         334822
numero_cuenta        23537
fecha_activacion     24034
mes_activacion       24034
camp_201902         334822
camp_201903         334822
camp_201904         334822
camp_201905         334822
camp_201906         334822
camp_201907         334822
camp_201908         334822
camp_201909         334822
camp_201910         334822
camp_201911         334822
camp_201912         334822
camp_202001         334822
camp_202002         334822
camp_202003         334822
camp_202004         334822
camp_202005         334822
camp_202006         334822
camp_202007         334822
camp_202008         334822
camp_202009         334822
camp_202010         334822
camp_202011         334822
camp_202012         334822
camp_202101         334822
camp_202102         334822
camp_202103         334822
camp_202104         334822
c

In [60]:
# Número de veces que ha sido incluido en la campaña últimos x meses
# Número de meses desde que fue incluido en la campaña de adquisición por última vez
# Número de activaciones de TC trad últimos x meses
# Meses desde la última activación
# Número de veces que ha sido incluido en la campaña y no aceptado
# Si ya se ha aceptado anteriormente la oferta


In [61]:
base_final['mes_campaña'].value_counts().sort_index()

2019-01-01      488
2019-02-01     6547
2019-03-01     5129
2019-04-01     6818
2019-05-01     6921
2019-06-01     6876
2019-07-01     6005
2019-08-01     7209
2019-09-01     7385
2019-10-01     7130
2019-11-01     7463
2019-12-01     5560
2020-01-01     7502
2020-02-01     8663
2020-03-01    11366
2020-04-01     7750
2020-05-01    13010
2020-06-01     9972
2020-07-01    12024
2020-08-01    13122
2020-09-01    13672
2020-10-01    14961
2020-11-01    11893
2020-12-01    11976
2021-01-01    10166
2021-02-01    12278
2021-03-01     8773
2021-04-01    13660
2021-05-01     9455
2021-06-01    11054
2021-07-01     9443
2021-08-01     8933
2021-09-01    10182
2021-10-01     2364
2021-11-01     9979
2021-12-01     9967
2022-01-01     9126
Name: mes_campaña, dtype: int64

In [62]:
base_final.columns

Index(['cedula', 'FECHA', 'mes_campaña', 'TIPO CAMPAÑA', 'tipo_camp', 'venta',
       'camp_201901', 'numero_cuenta', 'fecha_activacion', 'mes_activacion',
       'camp_201902', 'camp_201903', 'camp_201904', 'camp_201905',
       'camp_201906', 'camp_201907', 'camp_201908', 'camp_201909',
       'camp_201910', 'camp_201911', 'camp_201912', 'camp_202001',
       'camp_202002', 'camp_202003', 'camp_202004', 'camp_202005',
       'camp_202006', 'camp_202007', 'camp_202008', 'camp_202009',
       'camp_202010', 'camp_202011', 'camp_202012', 'camp_202101',
       'camp_202102', 'camp_202103', 'camp_202104', 'camp_202105',
       'camp_202106', 'camp_202107', 'camp_202108', 'camp_202109',
       'camp_202110', 'camp_202111', 'camp_202112', 'camp_202201'],
      dtype='object')

In [63]:
base_final.dtypes

cedula                       int64
FECHA               datetime64[ns]
mes_campaña         datetime64[ns]
TIPO CAMPAÑA                object
tipo_camp                   object
venta                      float64
camp_201901                float64
numero_cuenta               object
fecha_activacion    datetime64[ns]
mes_activacion      datetime64[ns]
camp_201902                float64
camp_201903                float64
camp_201904                float64
camp_201905                float64
camp_201906                float64
camp_201907                float64
camp_201908                float64
camp_201909                float64
camp_201910                float64
camp_201911                float64
camp_201912                float64
camp_202001                float64
camp_202002                float64
camp_202003                float64
camp_202004                float64
camp_202005                float64
camp_202006                float64
camp_202007                float64
camp_202008         

In [64]:
pd.crosstab(base_final['mes_campaña'],base_final['venta'])

venta,0.0,1.0
mes_campaña,,
2019-01-01,421,67
2019-02-01,6119,428
2019-03-01,4832,297
2019-04-01,6308,510
2019-05-01,6326,595
2019-06-01,6485,391
2019-07-01,5635,370
2019-08-01,6760,449
2019-09-01,6875,510


## Agregando primer uso

In [65]:
dates = pd.Series(base_final['mes_campaña'].dt.strftime('%Y%m').unique().astype('int')).sort_values()

i = 0
for date in dates:
    print("Fecha:", date)
    data_temp = trx_masterfile(date)
    print(len(data_temp))
    
    if (len(data_temp) > 0):
        if i == 0:
            data_trx = data_temp.copy()
            del data_temp
            gc.collect()
            i = i + 1
        else:
            data_trx = pd.concat([data_trx,data_temp],ignore_index=True)

            del data_temp
            gc.collect()    

Fecha: 201901
Si existe la data trx masterfile para la fecha 201901
Se ha realizado el correcto tratamiento de la data masterfile periodo 201901
83034
Fecha: 201902
Si existe la data trx masterfile para la fecha 201902
Se ha realizado el correcto tratamiento de la data masterfile periodo 201902
77067
Fecha: 201903
Si existe la data trx masterfile para la fecha 201903
Se ha realizado el correcto tratamiento de la data masterfile periodo 201903
90183
Fecha: 201904
Si existe la data trx masterfile para la fecha 201904
Se ha realizado el correcto tratamiento de la data masterfile periodo 201904
90254
Fecha: 201905
Si existe la data trx masterfile para la fecha 201905
Se ha realizado el correcto tratamiento de la data masterfile periodo 201905
97640
Fecha: 201906
Si existe la data trx masterfile para la fecha 201906
Se ha realizado el correcto tratamiento de la data masterfile periodo 201906
84423
Fecha: 201907
Si existe la data trx masterfile para la fecha 201907
Se ha realizado el correct

In [66]:
print(data_trx.shape)
data_trx.head(15)

(4483736, 11)


,tipo_id,identificacion_cliente,nro_cuenta,numero_tarjeta,fecha_transaccion,valor_total,nombre_comercio,cuotas,tipo_tc,mcc,periodo
0,4,1000794322,4506584002748351436,450658******9736,2019-01-25,38620.25,SP * NOTEBOOKTHERAPY,1,Tradicional,5734,201901
1,4,1000794322,4506584002748351436,450658******9736,2019-01-03,13884.01,NINTENDO *AMERICACA,1,Tradicional,5732,201901
2,4,1000794322,4506584002748351436,450658******9736,2019-01-12,41557.83,ONLY FANS,1,Tradicional,5967,201901
3,4,1000794322,4506584002748351436,450658******9736,2019-01-12,31992.17,ONLY FANS,1,Tradicional,5967,201901
4,4,1000794322,4506584002748351436,450658******9736,2019-01-13,31992.17,ONLY FANS,1,Tradicional,5967,201901
5,4,1000794322,4506584002748351436,450658******9736,2019-01-13,54354.70,ONLY FANS,36,Tradicional,5967,201901
6,4,1000794322,4506584002748351436,450658******9736,2019-01-17,31874.60,ONLY FANS,1,Tradicional,5967,201901
7,4,1000794322,4506584002748351436,450658******9736,2019-01-22,34980.89,ONLY FANS,1,Tradicional,5967,201901
8,4,1000794322,4506584002748351436,450658******9736,2019-01-22,47712.78,ONLY FANS,1,Tradicional,5967,201901
9,4,1000794322,4506584002748351436,450658******9736,2019-01-14,40750.00,MERCADOPAGO COLOMBIA,3,Tradicional,5817,201901


In [67]:
base_final.head(20)

,cedula,FECHA,mes_campaña,TIPO CAMPAÑA,tipo_camp,venta,camp_201901,numero_cuenta,fecha_activacion,mes_activacion,camp_201902,camp_201903,camp_201904,camp_201905,camp_201906,camp_201907,camp_201908,camp_201909,camp_201910,camp_201911,camp_201912,camp_202001,camp_202002,camp_202003,camp_202004,camp_202005,camp_202006,camp_202007,camp_202008,camp_202009,camp_202010,camp_202011,camp_202012,camp_202101,camp_202102,camp_202103,camp_202104,camp_202105,camp_202106,camp_202107,camp_202108,camp_202109,camp_202110,camp_202111,camp_202112,camp_202201
0,79711820,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,20896135,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,1.0,1.0,5361706003729575037,2019-02-05,2019-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,20896135,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,1.0,1.0,4506584002622716431,2019-02-05,2019-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,41527303,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,79491343,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5,39702994,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
6,13809106,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
7,34973956,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
8,34543992,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
9,41492284,2019-01-04,2019-01-01,Preaprobados,of-preaprobados,0.0,1.0,NaN,NaT,NaT,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [68]:
base_final.reset_index([llave,'numero_cuenta'], inplace=True)

NameError: name 'llave' is not defined

In [ ]:
## Defining llave
# cedula
llave = 'cedula'
## Defining index
base_final.columns = base_final.columns.str.lower()
base_final.set_index([llave,'numero_cuenta'], inplace=True)

## Renaming variables
data_trx.rename(columns={'identificacion_cliente':'cedula',
                        'nro_cuenta':'numero_cuenta'}, inplace=True)

In [ ]:
data_trx['cedula'] = data_trx['cedula'].astype('int')


In [ ]:
act_dates = pd.Series(base_final.loc[base_final['venta'] == 1,'fecha_activacion'].unique()).sort_values()
act_dates

In [ ]:
print(base_final.reset_index().dtypes)
print(data_trx.dtypes)

In [ ]:
#### Calculando primer uso

for date in act_dates:
    print('Fecha Activación:',date)
    data_tx_temp = data_trx[data_trx['fecha_transaccion']>=date]
    base_final.loc[(base_final['venta'] == 1) & (base_final['fecha_activacion'] == date),'fecha_transaccion'] = data_tx_temp.groupby([llave,'numero_cuenta'])['fecha_transaccion'].min()
    del data_tx_temp

## Calculando días primer uso
base_final['dias_first_use'] = (base_final['fecha_transaccion'] - base_final['fecha_activacion'])/pd.offsets.Day(1)
base_final['30first_use'] = np.where(base_final['dias_first_use']<=30,1,0)
base_final.reset_index(inplace=True)

In [ ]:
base_final['fecha_transaccion'].isnull().sum()/base_final.shape[0]

In [ ]:
base_final['venta'] = base_final['venta'].fillna(0)
###
tabla_final = base_final.groupby(['cedula','tipo_camp','mes_campaña'])['venta','30first_use'].max().reset_index()
tabla_final = tabla_final.rename(columns={'cedula':'id_numero_cliente'})
tabla_final

In [ ]:
print(tabla_final['30first_use'].value_counts()/tabla_final.shape[0])
print(tabla_final.loc[tabla_final['venta']==1,'30first_use'].value_counts()/tabla_final[tabla_final['venta']==1].shape[0])

In [ ]:
### Guardando Base:
path_final = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/base_VO_recalibración'

tabla_final.to_parquet(path_final, engine = 'pyarrow', index = False)

In [ ]:
#aaa

## Cargando MDTS  ###(si se requiere)

In [ ]:
path = 's3://adl-refined-dev-popular/parquet/TC_adquisicion'

objects = list_files(path)
files = [path + '/' + file for file in objects if ("total_tdc_M" in file)]
files = files[1:] # Se arranca desde abril
files

In [ ]:
for ix, file in enumerate(files):
    print(file)
    if ix == 0:
        base_mdt = spark_read_parquet(file)
        base_mdt = base_mdt.rename(columns={'sum_tx_linea':'sum_tx_linea_pasv', 'sum_tx_nolinea':'sum_tx_nolinea_pasv'})
        gc.collect()
    else:
        base_temp = spark_read_parquet(file)
        base_temp = base_temp.rename(columns={'sum_tx_linea':'sum_tx_linea_pasv', 'sum_tx_nolinea':'sum_tx_nolinea_pasv'})
        base_mdt = pd.concat([base_mdt,base_temp])
        del base_temp
        gc.collect()

In [ ]:
print(base_mdt.shape)
base_mdt.head(20)

In [ ]:
var_finales = [
'id_numero_cliente',
'mes_campaña',
'amortizacion_min_LB',#
'cuota_max_LB_SMLV',#
'dias_desde_ult_tx_ahr',#
'num_lib_solicitadas',#
'num_no_aceptado',#
'num_tx_ult_1mes_ahr',#
'numero_obligaciones_activasdif',#
'pasv_antig_total',#
'pasv_saldo_ca_1mes_atras',#
'porcentaje_utilizacion',#
'prom_monto_novado',#
'prom_n_cuotas',#
'quanto_mod',#
'sum_tx_linea_ahr',#
'valor_cuotas_codeudores_smlv',#
'valor_utilizado_smlv',#
]

base_mdt = base_mdt.rename(columns = {'fecha_camp':'mes_campaña',
                                      'amortizacion_min_lb':'amortizacion_min_LB',
                                      'cuota_max_lb_smlv':'cuota_max_LB_SMLV'})
base_mdt = base_mdt[var_finales]
base_mdt['mes_campaña'] = pd.to_datetime(base_mdt['mes_campaña'], format='%Y%m')

## Cargando Score:

In [ ]:
### Cargando Score:

files = [
'propension-tc_preaprob_M20210318.csv',
'propension-tc_preaprob_M20210419.csv',
'propension-tc_preaprob_M20210520.csv',
'propension-tc_preaprob_M20210616.csv',
'propension-tc_preaprob_M20210713.csv',
'propension-tc_preaprob_M20210811.csv',
'propension-tc_preaprob_M20210914.csv',
'propension-tc_preaprob_M20211014.csv',
'propension-tc_preaprob_M20211112.csv'
]

fechas = ['2021-04-01', '2021-05-01', '2021-06-01', '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01']

In [ ]:
path = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/'

i = 0
for file, date in zip(files, fechas):
    if i == 0:
        base_score = pd.read_csv(path+file, sep='|')
        base_score['mes_campaña'] = date
        base_score['id_numero_cliente'] = base_score['id_numero_cliente'].astype('int')
        i = i+1
    else:
        base_temp = pd.read_csv(path+file, sep='|')
        base_temp['mes_campaña'] = date
        base_temp['id_numero_cliente'] = base_temp['id_numero_cliente'].astype('int')
        base_score = pd.concat([base_score,base_temp])
        
        del base_temp
        gc.collect()


In [ ]:
base_score.columns

In [ ]:
base_final.columns

In [ ]:
base_final['mes_campaña'].value_counts().sort_index()

In [ ]:
base_score['mes_campaña'] = pd.to_datetime(base_score['mes_campaña'])

## Join 

In [ ]:
base_final = base_final.rename(columns={'cedula':'id_numero_cliente'})

In [ ]:
print(base_final[base_final['mes_campaña']>'2021-03-01'].shape)  
print(base_final.shape)  

In [ ]:
base_final.head()

In [ ]:
base_score.head()

In [ ]:
base_final = base_final[(base_final['mes_campaña']>'2021-03-01') & (base_final['mes_campaña']<'2021-11-01')]

In [ ]:
tabla_final = pd.merge(base_final[['id_numero_cliente', 'mes_campaña', 'tipo_camp', 
                                   'fecha_activacion', 'mes_activacion','venta',
                                  'fecha_transaccion','30first_use']],
                       base_score[['id_numero_cliente', 'mes_campaña', 'score', 'decil']], 
                       on=['id_numero_cliente', 'mes_campaña'], how='left')


print(base_score.shape)
print(base_final.shape)
print(tabla_final.shape)

tabla_final.head()

In [ ]:
base_mdt

In [ ]:
## SI SE REQUIERE
tabla_finalF = pd.merge(tabla_final,base_mdt, 
                       on=['id_numero_cliente', 'mes_campaña'], how='left')


print(tabla_final.shape)
print(base_mdt.shape)
print(tabla_finalF.shape)

tabla_finalF.head()

In [ ]:
tabla_finalF.count()

In [ ]:
tabla_finalF['30first_use'].value_counts()

## Guardando base MDT

In [ ]:
periodo = '202111'
path_final = 's3://adl-refined-dev-popular/parquet/TC_adquisicion/MDT_recalib_M'

tabla_finalF.to_parquet(path_final+periodo, engine = 'pyarrow', index = False)

## Corriendo Backtesting


In [ ]:
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler

In [ ]:
## Funcion para Backtesting

def backtesting(data,var_probab,var_target,cant,cutoff):

    dataframe = data.copy()
    dataframe[var_probab] = round(dataframe[var_probab]*1000)

    if np.isnan(cutoff).any():
        dataframe['rangos'] = pd.qcut(dataframe[var_probab],cant,precision=0,duplicates='drop')
    else:
        dataframe['rangos'] = pd.cut(dataframe[var_probab],cutoff,precision=0,include_lowest=True)
    
    dataframe['target'] = dataframe[var_target]
    
    table = pd.DataFrame(dataframe[['rangos','target']].groupby('rangos').size())
    table = table.sort_values(by ='rangos', ascending=True)
    table.rename(columns={0: 'total cat'}, inplace=True)

    table = table.join(dataframe[['rangos','target']].groupby(['rangos']).sum())
    table.rename(columns={'target': 'var 1'}, inplace=True)
    table['var 0'] = table['total cat'] - table['var 1']
    
    table.insert(loc=1, column='% total cat', value=round((table['total cat'] / dataframe.shape[0])*100,2))
    table.insert(loc=2, column='total acum', value=table['% total cat'].cumsum())
    
    table.insert(loc=4, column='tasa var 1', value=round((table['var 1']/table['total cat'])*100,2))
    table.insert(loc=6, column='tasa var 0', value=round((table['var 0']/table['total cat'])*100,2))
    
    table.insert(loc=5, column='var 1 acum', value=round((table['var 1'].cumsum()/table['var 1'].sum())*100,2))
    table.insert(loc=8, column='var 0 acum', value=round((table['var 0'].cumsum()/table['var 0'].sum())*100,2))
    
    table['KS'] = round(abs(table['var 1 acum']-table['var 0 acum']),2)
    
    table['tasa var 0 acum'] = [round((table.iloc[0+i:len(table),table.columns.get_loc('var 0')].sum()/
                                       table.iloc[0+i:len(table),table.columns.get_loc('total cat')].sum())*100,2) for i in range(0,len(table))]
    
    table['odds acum'] = [round((table.iloc[0+i:len(table),table.columns.get_loc('var 1')].sum()/
                                 table.iloc[0+i:len(table),table.columns.get_loc('var 0')].sum()),2) for i in range(0,len(table))]
    
    table['odds'] = round(table['var 1']/table['var 0'],2)
    
    comp = pd.DataFrame([int(np.where(table.iloc[1+i,table.columns.get_loc('odds')]>= \
                                      table.iloc[0+i,table.columns.get_loc('odds')],0,1)) for i in range(0,len(table)-1)])
    
    table['quiebres'] = pd.concat([pd.DataFrame([0]),comp]).set_index(table.index)
     
    table = table.sort_values(by ='rangos', ascending=False)
    
    table.reset_index(inplace=True)
    
    last_row = pd.DataFrame(['total',
                             table['total cat'].sum(),
                             round(table['% total cat'].sum()),
                             '',
                             table['var 1'].sum(),
                             round((table['var 1'].sum()/table['total cat'].sum())*100,2),
                             '',
                             table['var 0'].sum(),
                             round((table['var 0'].sum()/table['total cat'].sum())*100,2),
                             '',
                             table['KS'].max(),
                             '',
                             '',
                             round(table['var 1'].sum()/table['var 0'].sum(),2),
                             table['quiebres'].sum()]).T
    
    last_row.columns = table.columns

    table = table.append(last_row)
    
    return table




In [ ]:
pd.crosstab(tabla_final['decil'],tabla_final['venta'])

In [ ]:
tabla = backtesting(tabla_final,'score','venta',5,cutoff = np.nan)
tabla

In [ ]:
### Estabilidad
tabla_final['decil_mod']=pd.qcut(tabla_final['score'].rank(method='first'),10, labels=False)
pd.crosstab(tabla_final['decil'],tabla_final['mes_campaña'])

In [ ]:
pd.pivot_table(tabla_final, index='decil', columns=['mes_campaña','venta'], aggfunc='count')['FECHA']

In [ ]:
aaaaaa

### Guardando Base:

In [ ]:
path_final+periodo

In [ ]:
aaaaaa